## Search experiments

In [1]:
import requests

In [2]:
r=requests.get(f'https://inspirehep.net/api/experiments?size={240}')

In [42]:
#r=requests.get('https://inspirehep.net/api/experiments?q=CERN-LHC-CMS')

In [3]:
i=1
lex=r.json().get('hits').get('hits')
#lex[i].get('id'),lex[i].get('metadata').get('legacy_name')

In [4]:
j=1
lex=r.json().get('hits').get('hits')
#While in lex

### Search experiment
By control number

In [ ]:
ex=lex[j]
control_number=ex.get('id')
#Get Experiment authors profiles
#...
# Get Experiment papers

In [5]:
control_number=1108642
#temporal ex
url_exp=f'https://inspirehep.net/api/experiments/{control_number}'
r=requests.get(url_exp)
ex=r.json().get('metadata')
JSON(ex)

## Define class

In [ ]:
class author(profile):
    sleep = 0.4
    #`ids → [{"schema": "INSPIRE BAI"}]` required for author_id
    #`affiliations → [{...}] required for institution_id
    schema_author={"ids": {'type':'list',
                           'schema':{'type':'dict',
                                     'schema':{'schema':{'type':'string'},
                                               'value': {'type':'string'}
                                              }
                                    }
                          }, 
            "record":{'type':'dict'}, 
            "full_name": {'type':'string'}, 
            "affiliations":{'type':'list',
                           'schema':{'type':'dict','required':True}}
           }
    sample_author={"ids":[{"value": "D.Restrepo.1", 
                            "schema": "INSPIRE BAI"}
                          ], 
                   "uuid": "300c9b2c-15b0-4937-ad36-8ccff33fd09d", 
                   "emails": ["restrepo@udea.edu.co"], 
                   "record": {"$ref": "https://inspirehep.net/api/authors/991924"}, 
                   "full_name": "Restrepo, Diego", 
                   "affiliations": [{"value": "Antioquia U.", 
                                     "record": {"$ref": "https://inspirehep.net/api/institutions/903906"}},
                                   {"value":"Campinas State U.",
                                    "record":{"$ref":"https://inspirehep.net/api/institutions/902714"}}], 
                   "signature_block": "RASTRAPd", 
                   "raw_affiliations": [{"value": "Instituto de Física, Universidad de Antioquia, Calle 70 No 52-21, Medellín, Colombia"}, 
                                        {"value": "Instituto de Física Gleb Wataghin, UNICAMP, 13083-859, Campinas, SP, Brazil"}]}
    def __init__(self,a,db=[]):
        self.author=a
        self.db=db
        if a:
            v=Validator(self.schema_author,allow_unknown=True)
            if not v.validate(a):
                raise Exception(f'''
                    Input is not an INSPIRE-HEP author dictionary:
                    {v.errors}
                    See `self.sample_author`''')
        #if not hasattr(self,'cache'):
        #    self.cache={}
        #if not hasattr(self,'cachecho'):
        #    self.cacheco={}                        
        #super(author, self).__init__(p)
    def get_authors(self):
        #TODO: Check previous analysis for more metadata
        #use requests
        a=self.author
        r=empty_json()
        self.full_name=a.get('full_name')
        try:
            self.author_id=[i for i  in a.get('ids') if i.get('schema')=='INSPIRE BAI'
                  ][0].get('value')
        except:
            self.author_id=None

        #a.get('affiliations') is not always there
        try:
            affiliations=[dict([(k,d.get(k)) for k in d if k in ['value','record']]) 
                               for d in a.get('affiliations') ]
        except TypeError:
            affiliations=[]
            
        #Profile
        try:
            url=a.get('record').get("$ref")
        except AttributeError:
            url=''
        if url and isinstance(url,str):
            self.profile_id=url.split('/')[-1]
            r=empty_json()
            try:
                r=requests.get(url,timeout=timeout)
                time.sleep(self.sleep)
            except:
                r.status_code=-1
        if r.status_code==200:
            p=r.json().get('metadata')
        else:
            p={}#{'status_code':r.status_code}
            #return self.db #author without affilitions are not considered
            
        #We assume that the affilition is defined at least for
        #one of the authors of the paper
        #Authos without affiliations to get their institution are not considered
        super(author, self).__init__(p)             
        d=super(author, self).get_author() #→ self.profile
        if self.author_id is None:
            try:
                self.author_id=[i for i  in self.profile.get('ids') if i.get('schema')=='INSPIRE BAI'
                  ][0].get('value')
            except:
                self.author_id=None            
            
        
        #self.to_dict=super(author, self).to_dict
        ll=[x for x in self.db if hasattr(x,'author_id') and hasattr(x,'institution_id')]
        #self.db[i] affiliation not in affiliation are not touched
        self.institution=''
        self.institution_id=None
        self.institution_ids=[]
        for aff in affiliations:
            #We assumme that aff have at least the institution infomation
            self.institution=aff.get('value')
            if not self.institution:
                self.institution_id=None
                self.country=None
                continue
            try:
                aff_url=aff.get('record').get('$ref')            
                self.institution_id=aff_url.split('/')[-1]
                self.institution_ids.append(self.institution_id)
                #self.cache[self.institution]=self.institution_id
            except AttributeError:
                self.institution_id=None#self.cache.get(self.institution) #None if not key
                aff_url=None
            
            #if self.cacheco.get(self.institution):
            #    self.country=self.cacheco.get(self.institution)
            self.country=None
            if aff_url:
                rr=empty_json()
                try:
                    rr=requests.get(aff_url,timeout=timeout)
                    time.sleep(self.sleep)
                except:
                    rr.status_code=-1
                if rr.status_code==200:
                    try:
                        self.country=rr.json().get('metadata').get('addresses')[0].get('country')
                        #if self.country:
                        #    self.cacheco[self.institution]=self.country
                    except:
                        self.country=None
            else:
                self.country=None
                
            
            filtered_db=[x for x in ll if x.author_id==self.author_id 
                         and x.institution_id==self.institution_id]
            if filtered_db:
                au=filtered_db[0] #must be unique!
            else: #aff not in self.db → New affiliation
                ai=copy.copy(self)#deepcopy(self)
                self.db.append(ai)
                del ai #be sure that ai will not be modified
        return self.db
    def to_json(self):
        return [d.to_dict() for d in self.db]


#### Obtain papers metadata

In [10]:
paper={}
paper['legacy_name']=ex.get('legacy_name')
paper['control_number']=ex.get('control_number')
paper['number_of_papers']=ex.get('number_of_papers')
paper['collaboration']=ex.get("collaboration").get('value')
paper['literature_link']=f"https://inspirehep.net/api/literature?size=25&page=1&q=collaboration:{paper.get('collaboration')}"

In [11]:
paper

{'legacy_name': 'CERN-LHC-CMS',
 'control_number': 1108642,
 'number_of_papers': 13049,
 'collaboration': 'CMS',
 'literature_link': 'https://inspirehep.net/api/literature?size=25&page=1&q=collaboration:CMS'}

#### Obtain authors metadata

In [12]:
#TODO: Load papers for existing author
papers=[]

In [ ]:
paper={}
paper['legacy_name']=ex.get('legacy_name')
paper['control_number']=ex.get('control_number')
paper['number_of_papers']=ex.get('number_of_papers')
paper['collaboration']=ex.get("collaboration").get('value')
paper['literature_link']=f"https://inspirehep.net/api/literature?size=25&page=1&q=collaboration:{paper.get('collaboration')}"

url=f'https://inspirehep.net/api/authors?q=project_membership.record.%24ref%3A{control_number}&page=1&size=25'
r=requests.get(url)

pp=r.json().get('hits').get('hits')[0].get('metadata') #experimental author

In [13]:
url=f'https://inspirehep.net/api/authors?q=project_membership.record.%24ref%3A{control_number}&page=1&size=25'
r=requests.get(url)

In [14]:
url

'https://inspirehep.net/api/authors?q=project_membership.record.%24ref%3A1108642&page=1&size=25'

In [33]:
pp=r.json().get('hits').get('hits')[0].get('metadata')

In [34]:
pp

{'project_membership': [{'name': 'CERN-LHC-CMS',
   'record': {'$ref': 'https://inspirehep.net/api/experiments/1108642'},
   'current': False,
   'curated_relation': True},
  {'name': 'CERN-NA64',
   'record': {'$ref': 'https://inspirehep.net/api/experiments/1108642'},
   'current': False,
   'curated_relation': True}],
 'positions': [{'rank': 'SENIOR',
   'record': {'$ref': 'https://inspirehep.net/api/institutions/903030'},
   'current': True,
   'institution': 'Moscow, INR',
   'curated_relation': True},
  {'rank': 'PHD',
   'record': {'$ref': 'https://inspirehep.net/api/institutions/903030'},
   'end_date': '1988',
   'institution': 'Moscow, INR',
   'curated_relation': True}],
 'ids': [{'value': '0000-0002-7397-9665', 'schema': 'ORCID'},
  {'value': 'Y.Andreev.1', 'schema': 'INSPIRE BAI'},
  {'value': 'INSPIRE-00320310', 'schema': 'INSPIRE ID'},
  {'value': 'HEPNAMES-412040', 'schema': 'SPIRES'}],
 'name': {'value': 'Andreev, Yuri',
  'native_names': ['Юрий Андреев'],
  'name_varia

In [19]:
url=f"https://inspirehep.net/api/authors/{pp.get('control_number')}"
rp=requests.get(url)

In [20]:
class experiment(profile):
    sleep=0.4
    def __init__(self,*args, **kwargs):
        super(experiment, self).__init__(*args, **kwargs)
        
    

In [21]:
p=rp.json().get('metadata')

In [9]:
JSON(p)

<IPython.core.display.JSON object>

In [22]:
e=experiment(p)

In [23]:
self=e.get_author()

In [24]:
JSON(self.to_dict())

<IPython.core.display.JSON object>

In [25]:
self.full_name=p.get('name').get('value')
self.profile_id=p.get('control_number')
#url.split('/')[-1]

In [26]:
JSON(self.profile)

<IPython.core.display.JSON object>

In [27]:
self.full_name=p.get('name').get('value')
self.profile_id=p.get('control_number')
try:
    self.author_id=[i for i  in self.profile.get('ids') if i.get('schema')=='INSPIRE BAI'
                  ][0].get('value')
except:
    self.author_id=None 
self.author_id

'Y.Andreev.1'

In [28]:
self.full_name=p.get('name').get('value')
self.profile_id=p.get('control_number')
try:
    self.author_id=[i for i  in self.profile.get('ids') if i.get('schema')=='INSPIRE BAI'
                  ][0].get('value')
except:
    self.author_id=None 
#try
current_position=self.profile.get('positions')[0]
self.institution=current_position.get('institution')
self.institution

'Moscow, INR'

In [29]:
self.full_name=p.get('name').get('value')
self.profile_id=p.get('control_number')
try:
    self.author_id=[i for i  in self.profile.get('ids') if i.get('schema')=='INSPIRE BAI'
                  ][0].get('value')
except:
    self.author_id=None 
#try
current_position=self.profile.get('positions')[0]
self.institution=current_position.get('institution')
aff_url=current_position.get('record').get('$ref')            
self.institution_id=aff_url.split('/')[-1]
self.institution_id

'903030'

In [30]:
#TODO: Move to class!
self.country=None
if aff_url:
    rr=empty_json()
    try:
        rr=requests.get(aff_url,timeout=timeout)
        time.sleep(self.sleep)
    except:
        rr.status_code=-1
    if rr.status_code==200:
        try:
            self.country=rr.json().get('metadata').get('addresses')[0].get('country')
            # if self.country:
            #    self.cacheco[self.institution]=self.country
        except:
            self.country=None
else:
    self.country=None
self.country

'Russian Federation'

In [31]:
#repeat for each author
papers.append(paper)
self.papers=papers

In [32]:
JSON(self.to_dict())

<IPython.core.display.JSON object>

## Define class

In [2]:
from inspirehep import *
class experimental_author(profile):
    sleep = 0.4
    #`ids → [{"schema": "INSPIRE BAI"}]` required for author_id
    #`affiliations → [{...}] required for institution_id
    schema_experimental_author={"ids": {'type':'list',
                           'schema':{'type':'dict',
                                     'schema':{'schema':{'type':'string'},
                                               'value': {'type':'string'}
                                              }
                                    }
                          }, 
            "record":{'type':'dict'}, 
            "full_name": {'type':'string'}, 
            "affiliations":{'type':'list',
                           'schema':{'type':'dict','required':True}},
            "proyect_membership":{'type':'list',
                                    'schema':{'type':'dict',
                                              'schema':{'record':{'type':'dict'},
                                                        'name':{'type':'string'},
                                                        'current':{'type':'boolean'}
                                                       }
                                             }
                                },
            "name":{'type':'dict',
                    'schema':{'value':{'type':'string'},
                              'preferred_name':{'type':'string'},
                              }
                    },
            "control_number":{'type':'integer'},
            "arxiv_categories":{'type':'list'}
           }
    sample_experimental_author={'project_membership': [{'name': 'CERN-LHC-CMS',
                                    'record': {'$ref': 'https://inspirehep.net/api/experiments/1108642'},
                                    'current': False,
                                    'curated_relation': True}],
                                'ids': [{'value': 'Y.Andreev.1', 'schema': 'INSPIRE BAI'}],
                                'name': {'value': 'Andreev, Yuri',
                                'preferred_name': 'Yuri Andreev'},
                                'control_number': 1018372,
                                'arxiv_categories': ['hep-ex']
                                }
    

    def __init__(self,a,db=[]):
        self.author=a
        self.db=db
        if a:
            v=Validator(self.schema_experimental_author,allow_unknown=True)
            if not v.validate(a):
                raise Exception(f'''
                    Input is not an INSPIRE-HEP author dictionary:
                    {v.errors}
                    See `self.sample_author`''')
    def get_authors(self):
        #TODO: Check previous analysis for more metadata
        #use requests
        # Not longer required

        a=self.author


        self.project_membership = a.get('project_membership')
        control_number = a.get('control_number')
        url = f"https://inspirehep.net/api/authors/{control_number}"
        r=empty_json()

        try:
            r=requests.get(url,timeout=timeout)
            time.sleep(self.sleep)
        except:
            r.status_code=-1
        if r.status_code==200:
            p=r.json().get('metadata')
        else:
            p={}
        #We assume that the affilition is defined at least for
        #one of the authors of the paper
        #Authos without affiliations to get their institution are not considered
        super(experimental_author, self).__init__(p)             
        d=super(experimental_author, self).get_author() #→ self.profile
        self.full_name=p.get('name').get('value')
        self.profile_id=p.get('control_number')
        try:
            self.author_id=[i for i  in self.profile.get('ids') if i.get('schema')=='INSPIRE BAI'
                        ][0].get('value')
        except:
            self.author_id=None 

        ll=[x for x in self.db if hasattr(x,'author_id') and hasattr(x,'institution_id')]
        # TODO: Check if the author has multiple current positions
        # for positions...
        try:
            current_position=self.profile.get('positions')[0]
        except:
            current_position={}
        self.institution=current_position.get('institution')
        try:
            aff_url=current_position.get('record').get('$ref')
        except:
            aff_url=''
        self.institution_id=aff_url.split('/')[-1]
        # ==== country ====
        self.country=None
        if aff_url:
            rr=empty_json()
            try:
                rr=requests.get(aff_url,timeout=timeout)
                time.sleep(self.sleep)
            except:
                rr.status_code=-1
            if rr.status_code==200:
                try:
                    self.country=rr.json().get('metadata').get('addresses')[0].get('country')
                    # if self.country:
                    #    self.cacheco[self.institution]=self.country
                except:
                    self.country=None
        else:
            self.country=None
        # =================
        filtered_db=[x for x in ll if x.author_id==self.author_id 
                         and x.institution_id==self.institution_id]
        if filtered_db:
            au=filtered_db[0] #must be unique!
        else: #aff not in self.db → New affiliation
            ai=copy.copy(self)#deepcopy(self)
            self.db.append(ai)
            del ai #be sure that ai will not be modified

        return self.db
    def to_json(self):
        return [d.to_dict() for d in self.db]


class experiment(experimental_author):
    '''
    paper['legacy_name']=ex.get('legacy_name')
    paper['control_number']=ex.get('control_number')
    paper['number_of_papers']=ex.get('number_of_papers')
    paper['collaboration']=ex.get("collaboration").get('value')
    paper['literature_link']
    '''
    schema_experiment={'control_number':{'type':'integer','required':True},
                       'number_of_papers':{'type':'integer'},
                       'legacy_name':{'type':'string'},
                       'collaboration':{'type':'dict',
                                        'schema':{'value':{'type':'string'}}},
                       'description':{'type':'string'},
                       'accelerator':{'type':'dict',
                                      'schema':{'value':{'type':'string'}}},
                       'date_approved':{'type':'string'},
                       'date_proposed':{'type':'string'},
                       'date_started':{'type':'string'},
                       'date_completed':{'type':'string'},
                       'project_type':{'type':'list'},
                       'inspire_classification':{'type':'list'}
            }
    def __init__(self,e,db=[],size=250):
        self.sample_experiment = {'number_of_papers': 13049, #
                                    'self': {'$ref': 'https://inspirehep.net/api/experiments/1108642'}, 
                                    'long_name': 'CMS: The Compact Muon Solenoid',
                                    'accelerator': {'value': 'LHC'},
                                    'description': 'The 27-km Large Hadron Collider (LHC) is the largest and most powerful particle accelerator ever built. It accelerates protons to nearly the velocity of light -- in clockwise and anti-clockwise directions -- and then collides them at four locations around its ring. At these points, the energy of the particle collisions gets transformed into mass, spraying particles in all directions. The Compact Muon Solenoid (or CMS) detector sits at one of these four collision points. It is a general-purpose detector; that is, it is designed to observe any new physics phenomena that the LHC might reveal. CMS acts as a giant, high-speed camera, taking 3D “photographs” of particle collisions from all directions up to 40 million times each second. Although most of the particles produced in the collisions are “unstable”, they transform rapidly into stable particles that can be detected by CMS. By identifying (nearly) all the stable particles produced in each collision, measuring their momenta and energies, and then piecing together the information of all these particles like putting together the pieces of a puzzle, the detector can recreate an “image” of the collision for further analysis.',
                                    'legacy_name': 'CERN-LHC-CMS',
                                    'date_approved': '1996-01-31', #
                                    'date_proposed': '1992-10-01', #
                                    'date_started': '2009-11-23', ##
                                    'date_completed': '9999', #
                                    'project_type': ['experiment'],
                                    'collaboration': {'value': 'CMS'}, #
                                    'control_number': 1108642, #
                                    'inspire_classification': ['Collider Experiments|Hadrons|p p'] #
                                    }
        if e:
            v=Validator(self.schema_experiment,allow_unknown=True)
            if not v.validate(e):
                raise Exception(f'''
                    Input is not an INSPIRE-HEP experiment dictionary:
                    {v.errors}
                    See `self.sample_work`''')
        
        self.db = db
        self.experiment = e
        self.authors_size = size

        
    def get_authors(self):
        '''
        l: list of author objects
        '''

        l=self.experiment

        print(l.get('control_number'),end='\r')
        #TODO: Check previous analysis for more metadata
        inspire_classification=l.get('inspire_classification')
        if not inspire_classification:
            inspire_classification = []
        
        control_number=l.get('control_number')

        try:
            year=int(l.get('date_started').split('-')[0])
        except TypeError:
            year=0
        if not year:
            year=9999
        #'inst_id':aff_id
        paper={}
        paper['legacy_name'] = l.get('legacy_name')
        paper['control_number'] = control_number
        paper['number_of_papers']=l.get('number_of_papers')
        paper['collaboration']=l.get("collaboration").get('value')
        paper['literature_link']=f"https://inspirehep.net/api/literature?size=25&page=1&q=collaboration:{paper.get('collaboration')}"
        paper['year']=year
        paper['inspire_classification']=inspire_classification
        paper['dates'] = {'approved':l.get('date_approved'),
                          'proposed':l.get('date_proposed'),
                          'started':l.get('date_started'), 
                          'completed':l.get('date_completed')}
        paper['long_name']=l.get('long_name')
        paper['description']=l.get('description')
        try:
            paper['accelerator']=l.get('accelerator').get('value')
        except:
            paper['accelerator']=None
        paper['project_type']=l.get('project_type')      


        #In update
        #'primary_arxiv_category':primary_arxiv_category
        #super(work, self).__init__(w.get('authors')[0])

        r=empty_json()
        # TODO: load all pages
        NEXT=True
        i=0        
        while NEXT:
            if i==0:
                url=f'https://inspirehep.net/api/authors?q=project_membership.record.%24ref%3A{control_number}&page=1&size={self.authors_size}'
            print(i,url)
            try:
                r=requests.get(url,timeout=timeout)
                time.sleep(self.sleep)
            except:
                r.status_code=-1
            if r.status_code==200:
                aus=r.json().get('hits').get('hits')
            else:
                aus=[]
            if i==0 and len(aus)==250:
                paper['description']=paper['description'][:20]+'...'
            for a in aus: #same self.author_id but several institute_ids for several affiliations
                links=r.json().get('links')
                total=r.json().get('hits').get('total')                
                super(experiment, self).__init__(a.get('metadata'),self.db)
                super(experiment, self).get_authors() #add and replace self attributes → author*..., institution*..
                paper['author_id']=self.author_id
                paper['profile_id']=self.profile_id
                #Each d object keeps its RAM memory space independent of reasignation list from db to adb
                adb=[d for d in self.db if d.author_id==self.author_id]

                papers = []
                for d in adb:
                    if hasattr(d,'papers'):
                        papers=d.papers
                        break #found papers for self.author_id
                for d in adb:
                    d.papers=papers #reatach papers if already have
                    if paper not in d.papers:
                        #detach from RAM!
                        cppaper=copy.copy(paper)
                        d.papers.append(cppaper)
                        del cppaper
                if links.get('next'):
                    url=links.get('next')
                else:
                    NEXT=False
                #EMERGENCY EXIT: We assume here that r.json().get('total') exists
                if i>2:
                    NEXT=False
                if i > total//self.authors_size+1:
                    NEXT=False
                i+=1            

        return self.db



In [3]:

sample_experimental_author={'project_membership': [{'name': 'CERN-LHC-CMS',
                                    'record': {'$ref': 'https://inspirehep.net/api/experiments/1108642'},
                                    'current': False,
                                    'curated_relation': True}],
                                'ids': [{'value': 'Y.Andreev.1', 'schema': 'INSPIRE BAI'}],
                                'name': {'value': 'Andreev, Yuri',
                                'preferred_name': 'Yuri Andreev'},
                                'control_number': 1018372,
                                'arxiv_categories': ['hep-ex']
                                }
a=experimental_author(sample_experimental_author)
db=a.get_authors()


In [4]:
sample_experiment = {'number_of_papers': 13049, #
                                    'self': {'$ref': 'https://inspirehep.net/api/experiments/1108642'}, 
                                    'long_name': 'CMS: The Compact Muon Solenoid',
                                    'accelerator': {'value': 'LHC'},
                                    'description': 'The 27-km Large Hadron Collider (LHC) is the largest and most powerful particle accelerator ever built. It accelerates protons to nearly the velocity of light -- in clockwise and anti-clockwise directions -- and then collides them at four locations around its ring. At these points, the energy of the particle collisions gets transformed into mass, spraying particles in all directions. The Compact Muon Solenoid (or CMS) detector sits at one of these four collision points. It is a general-purpose detector; that is, it is designed to observe any new physics phenomena that the LHC might reveal. CMS acts as a giant, high-speed camera, taking 3D “photographs” of particle collisions from all directions up to 40 million times each second. Although most of the particles produced in the collisions are “unstable”, they transform rapidly into stable particles that can be detected by CMS. By identifying (nearly) all the stable particles produced in each collision, measuring their momenta and energies, and then piecing together the information of all these particles like putting together the pieces of a puzzle, the detector can recreate an “image” of the collision for further analysis.',
                                    'legacy_name': 'CERN-LHC-CMS',
                                    'date_approved': '1996-01-31', #
                                    'date_proposed': '1992-10-01', #
                                    'date_started': '2009-11-23', ##
                                    'date_completed': '9999', #
                                    'project_type': ['experiment'],
                                    'collaboration': {'value': 'CMS'}, #
                                    'control_number': 1108642, #
                                    'inspire_classification': ['Collider Experiments|Hadrons|p p'] #
                                    }
e=experiment(sample_experiment,size=5)
db=e.get_authors()

0 https://inspirehep.net/api/authors?q=project_membership.record.%24ref%3A1108642&page=1&size=5
5 https://inspirehep.net/api/authors/?q=project_membership.record.%24ref%3A1108642&size=5&page=2
10 https://inspirehep.net/api/authors/?q=project_membership.record.%24ref%3A1108642&size=5&page=3
15 https://inspirehep.net/api/authors/?q=project_membership.record.%24ref%3A1108642&size=5&page=4


In [37]:
len(db)

250

In [14]:
rs=db[-1]

In [43]:
rs.profile_id

1004186

In [44]:
import json

In [51]:
e.to_json()

[{'author_id': 'Y.Andreev.1',
  'country': 'Russian Federation',
  'email_addresses': [{'value': 'andreyev@inr.ru', 'current': True},
   {'value': 'Yuri.Andreev@cern.ch', 'current': True}],
  'full_name': 'Andreev, Yuri',
  'ids': [{'value': '0000-0002-7397-9665', 'schema': 'ORCID'},
   {'value': 'Y.Andreev.1', 'schema': 'INSPIRE BAI'},
   {'value': 'INSPIRE-00320310', 'schema': 'INSPIRE ID'},
   {'value': 'HEPNAMES-412040', 'schema': 'SPIRES'}],
  'institution': 'Moscow, INR',
  'institution_id': '903030',
  'name': {'value': 'Andreev, Yuri',
   'native_names': ['Юрий Андреев'],
   'name_variants': ['Andreev, Yu.M.',
    'Andreev, Yury',
    'Andreev, Yurii',
    'Andreev, Iouri'],
   'preferred_name': 'Yuri Andreev'},
  'papers': [{'legacy_name': 'CERN-LHC-CMS',
    'control_number': 1108642,
    'number_of_papers': 13049,
    'collaboration': 'CMS',
    'literature_link': 'https://inspirehep.net/api/literature?size=25&page=1&q=collaboration:CMS',
    'year': 2009,
    'inspire_class

In [3]:
f=open('kk.json','w')
json.dump(e.to_json(),f)
f.close()


In [ ]:
dbla=[]
jsonla=[]
# loop in experiments
# obtain experiment:
# obtain db for experiment
# instersection of LA countries with experiment countries
# if intersection is not empty: dbla+db, jsonla+e.to_json()
